# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import os
import kagglehub
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

# Loading the dataset

In [3]:
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
csv_file = os.path.join(path, "IMDB Dataset.csv")
df = pd.read_csv(csv_file)

In [4]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Split the data into training, validation, and testing sets

In [5]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Load the tokenizer and encode the data

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_texts = train_data['review'].tolist()
train_labels = train_data['sentiment'].tolist()
train_labels = [1 if label == "positive" else 0 for label in train_labels]
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

val_texts = val_data['review'].tolist()
val_labels = val_data['sentiment'].tolist()
val_labels = [1 if label == "positive" else 0 for label in val_labels]
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

test_texts = test_data['review'].tolist()
test_labels = test_data['sentiment'].tolist()
test_labels = [1 if label == "positive" else 0 for label in test_labels]
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

# Convert the encoded inputs to TensorFlow datasets

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(10000).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(64)

# Load and fine-tune the model

In [8]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
2000/2000 [==============================] - 1921s 949ms/step - loss: 0.2454 - accuracy: 0.9014 - val_loss: 0.2207 - val_accuracy: 0.9109
Epoch 2/3
2000/2000 [==============================] - 1893s 947ms/step - loss: 0.1383 - accuracy: 0.9472 - val_loss: 0.2099 - val_accuracy: 0.9210
Epoch 3/3
2000/2000 [==============================] - 1894s 947ms/step - loss: 0.0722 - accuracy: 0.9754 - val_loss: 0.2061 - val_accuracy: 0.9273


# Evaluate the model on the testing set

In [9]:
results = model.evaluate(test_dataset)
print("test loss, test accuracy:", results)

157/157 [==============================] - 190s 1s/step - loss: 0.2044 - accuracy: 0.9290
test loss, test accuracy: [0.20442460477352142, 0.9290000200271606]


# Save the model

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
model.save('/content/drive/MyDrive/sentiment_model')
